# Import

In [1]:
import os
import numpy as np
import random
import time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_arg_scope

# Build graph

In [2]:
g = tf.get_default_graph()

## Define preprocessing

In [3]:
with tf.name_scope('Preprocessing'):
    input_im = tf.placeholder(tf.uint8, shape=[None, None, None, 3], name='input_im')
    resized_im = tf.image.resize_bilinear(tf.image.convert_image_dtype(
        tf.convert_to_tensor(input_im) ,dtype=tf.float32), [299,299], name='resized_im')
    normalized_im = tf.mul(tf.sub(resized_im, 0.5), 2.0, name='normalized_im')

## Load Inception-v3 graph from slim

In [4]:
with slim.arg_scope(inception_v3_arg_scope()):
    logits, end_points = inception_v3(normalized_im, num_classes=1001, is_training=False)
slim_variables = [v for v in tf.global_variables() if v.name.startswith('InceptionV3/')]

## Create own branch

In [5]:
with tf.name_scope('Own'):
    y = tf.placeholder("float")
    x = g.get_tensor_by_name('InceptionV3/Logits/Dropout_1b/Identity:0')
    w1 = tf.Variable(tf.random_normal([int(x.get_shape()[-1]),1000], stddev=float('1e-5')), name='weights_1')
    b1 = tf.Variable(tf.random_normal([1,1000]), name='bias_1')
    a1 = tf.add(tf.matmul(tf.squeeze(x), w1), b1, name='a_1')
    z1 = tf.nn.relu(a1, name='relu_1')
    w2 = tf.Variable(tf.random_normal([int(w1.get_shape()[-1]),300], stddev=float('1e-5')), name='weights_2')
    b2 = tf.Variable(tf.random_normal([1,300]), name='bias_2')
    y_pred = tf.add(tf.matmul(z1, w2), b2, name='y_pred')
    cost = tf.reduce_sum(tf.square(y-y_pred),name='cost')
    cost_summary = tf.summary.scalar('cost', cost)

# Read data

In [ ]:
# Images
val_image_path = '../../data/val2014/'
val_image_list = os.listdir(val_image_path)

# Feed through

In [ ]:
with tf.Session() as sess:
    
    # Initialize variables
#    timestamp = time.strftime('%Y%m%d-%H%M', time.localtime())
    checkpoint = '20170306-1722'
    saver = tf.train.Saver()
    latest_checkpoint = tf.train.latest_checkpoint(
        './slim_inception_v3/runs/' + checkpoint + '/checkpoint/')
    saver.restore(sess, latest_checkpoint)
    
    # Single-threaded
    new_vec_dict = {}
    for i, im in enumerate(val_image_list):
        temp_im = np.array(Image.open(val_image_path + val_image_list[i]))

        # If image has only one channel
        if len(np.shape(temp_im)) == 2:
            temp_im = np.stack([temp_im,temp_im,temp_im],axis=2)
            
        squeezed = sess.run(g.get_tensor_by_name('Own/Squeeze:0'), {input_im:[temp_im]})
        new_vec_dict[im] = sess.run(y_pred, {g.get_tensor_by_name('Own/Squeeze:0'):[squeezed]})
        if i % 100 == 0:
            print(i, end=' ')
    
    save_path = '../../data/image_space_' + checkpoint + '.npy'
    np.save(save_path,new_vec_dict)
    print('\n\nImage space saved to {}'.format(save_path))

0 